# Advanced MLP - 2
- Putting it altogether

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers

## Load Dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# reshaping X data: (n, 28, 28) => (n, 784)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1] * X_test.shape[2]))

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(60000, 784) (10000, 784) (60000,) (10000,)


## Training & Validating Model
- Measures to improve training is applied simultaneously
    - More training set
    - Weight Initialization scheme
    - Nonlinearity (Activation function)
    - Optimizers: adaptvie
    - Batch Normalization
    - Dropout (Regularization)
    - Model Ensemble

In [ ]:
def mlp_model():
    model = Sequential()
    
    model.add(Dense(50, input_shape = (784, ), kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))    
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, kernel_initializer='he_normal'))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
# create 5 models to ensemble
model1 = KerasClassifier(build_fn = mlp_model, epochs = 50)
model2 = KerasClassifier(build_fn = mlp_model, epochs = 50)
model3 = KerasClassifier(build_fn = mlp_model, epochs = 50)
model4 = KerasClassifier(build_fn = mlp_model, epochs = 50)
model5 = KerasClassifier(build_fn = mlp_model, epochs = 50)

model1._estimator_type = "classifier"
model2._estimator_type = "classifier"
model3._estimator_type = "classifier"
model4._estimator_type = "classifier"
model5._estimator_type = "classifier"

In [ ]:
ensemble_clf = VotingClassifier(estimators = [('model1', model1), ('model2', model2), ('model3', model3), ('model4', model4), ('model5', model5)]
                                , voting = 'soft')

In [ ]:
ensemble_clf.fit(X_train, y_train)

Epoch 1/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.7486 - accuracy: 0.7695
Epoch 2/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4274 - accuracy: 0.8761
Epoch 3/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3550 - accuracy: 0.8987
Epoch 4/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3179 - accuracy: 0.9088
Epoch 5/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2934 - accuracy: 0.9154
Epoch 6/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2796 - accuracy: 0.9204
Epoch 7/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2629 - accuracy: 0.9255
Epoch 8/50
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2527 - accuracy: 0.9275
Epoch 9/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2443 - accuracy: 0.9308
Epoch 10/50
1875/1875 [==============================] - 5s 2ms/step - loss: 0.235

VotingClassifier(estimators=[('model1',
                              <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f13a4fec550>),
                             ('model2',
                              <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f13d50c9f60>),
                             ('model3',
                              <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f13d50c9320>),
                             ('model4',
                              <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f13a59390f0>),
                             ('model5',
                              <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f13d50c9978>)],
                 flatten_transform=True, n_jobs=None, voting='soft',
                 weights=None)

In [ ]:
y_pred = ensemble_clf.predict(X_test)

Instructions for updating:
Please use `model.predict()` instead.


In [ ]:
print('Acc: ', accuracy_score(y_pred, y_test))

Acc:  0.9797


Final model produces around **98% of accuracy**, which is a dramatic improvement from the baseline model